In [ ]:
!pip install torch==1.11.0
!pip install torchtext==0.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.11.0 which is incompatible.
torchtext 0.16.2 requires torch==2.1.2, but you have torch 1.11.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.2
    Uninstalling torchtext-0.16.2:
      Successfully uninstalled torchtext-0.16.2


In [ ]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import time

In [55]:
start = time.time()
TEXT = torchtext.data.Field(lower=True, fix_length = 200, batch_first = False)
LABEL = torchtext.data.Field(sequential=False)

In [56]:
from torchtext import datasets

In [57]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [58]:
print(vars(train_data.examples[0]))

{'text': ['a', 'give', 'this', 'movie', 'a', '10/10', 'because', 'it', 'deserves', 'a', '10/10.', 'two', 'of', 'the', 'best', 'actors', 'of', 'their', 'time-walter', 'matthau', '&', 'george', 'burns', 'collaborate', 'with', 'neil', 'simon', 'and', 'all', 'of', 'the', 'other', 'actors', 'that', 'are', 'in', 'this', 'film', '+', 'director', 'herbert', 'ross,', 'and', 'all', 'of', 'that', 'makes', 'this', 'stage', 'adaption', 'come', 'true.', 'the', 'sunshine', 'boys', 'is', 'one', 'of', 'the', 'best', 'films', 'of', 'the', "70's.", 'i', 'love', 'the', 'type', 'of', 'humor', 'in', 'this', 'film,', 'it', 'just', 'makes', 'me', 'laugh', 'so', 'hard.<br', '/><br', '/>i', 'got', 'this', 'movie', 'on', 'vhs', '3', 'days', 'ago', '(yes,', 'vhs', 'because', 'it', 'was', 'cheaper-only', '$3).', 'i', 'watched', 'it', 'as', 'soon', 'as', 'i', 'got', 'home,', 'but', 'i', 'had', 'to', 'watch', 'it', 'again', 'because', 'i', 'kept', 'missing', 'a', 'few', 'parts', 'the', 'first', 'time.', 'the', 'seco

In [59]:
import string

for example in train_data.examples:
  text = [x.lower() for x in vars(example)['text']]
  text = [x.replace("<br","") for x in text]
  text = [''.join(c for c in s if c not in string.punctuation) for s in text]
  text = [s for s in text if s]
  vars(example)['text'] = text

In [60]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(0), split_ratio=0.8)

In [61]:
print(f'number of train data : {len(train_data)}')
print(f'number of valid data : {len(valid_data)}')
print(f'number of test data : {len(test_data)}')

number of train data : 20000
number of valid data : 5000
number of test data : 25000


In [62]:
TEXT.build_vocab(train_data, max_size=10000, min_freq=10, vectors=None)
LABEL.build_vocab(train_data)

print(f'Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}')

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 3


In [63]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x79c78a5274f0>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


In [64]:
BATCH_SIZE = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

embeding_dim = 100
hidden_size = 300

train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

In [65]:
class RNNCell_Encoder(nn.Module):
  def __init__(self, input_dim, hidden_size):
    super(RNNCell_Encoder, self).__init__()
    self.rnn = nn.RNNCell(input_dim, hidden_size)
  def forward(self, inputs):
    bz = inputs.shape[1]
    ht = torch.zeros((bz, hidden_size))
    for word in inputs:
      ht = self.rnn(word, ht)
    return ht

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.em = nn.Embedding(len(TEXT.vocab.stoi), embeding_dim)
    self.rnn = RNNCell_Encoder(embeding_dim, hidden_size)
    self.fc1 = nn.Linear(hidden_size, 256)
    self.fc2 = nn.Linear(256, 3)

  def forward(self, x):
    x = self.em(x)
    x = self.rnn(x)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [66]:
model = Net()
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [67]:
def training(epoch, model, trainloader, validloader):
  correct = 0
  total = 0
  running_loss = 0

  model.train()
  for b in trainloader:
    x, y = b.text, b.label
    x, y = x.to(device), y.to(device)
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    with torch.no_grad():
      y_pred = torch.argmax(y_pred, dim=1)
      correct += (y_pred == y).sum().item()
      total += y.size(0)
      running_loss += loss.item()

    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct/total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
      for b in trainloader:
        x, y = b.text, b.label
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        y_pred = torch.argmax(y_pred, dim=1)
        valid_correct += (y_pred == y).sum().item()
        valid_total += y.size(0)
        valid_running_loss += loss.item()

  epoch_valid_loss = valid_running_loss / len(validloader.dataset)
  epoch_valid_acc = valid_correct / valid_total

  print('epoch: ', epoch,
        'loss: ', round(epoch_loss, 3),
        'accuracy: ', round(epoch_acc, 3),
        'valid_loss: ', round(epoch_valid_loss, 3),
        'valid_accuracy: ', round(epoch_valid_acc, 3))
  return epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc

In [68]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []
for epoch in range(epochs):
  epoch_loss, epoch_acc, epoch_valid_loss, epoch_valid_acc = training(epoch, model, train_iterator, valid_iterator)
  train_loss.append(epoch_loss)
  train_acc.append(epoch_acc)
  valid_loss.append(epoch_valid_loss)
  valid_acc.append(epoch_valid_acc)
end = time.time()
print(f'total time spend: {end-start}')

epoch:  0 loss:  0.0 accuracy:  0.062 valid_loss:  0.069 valid_accuracy:  0.379
epoch:  1 loss:  0.0 accuracy:  0.438 valid_loss:  0.068 valid_accuracy:  0.387
epoch:  2 loss:  0.0 accuracy:  0.391 valid_loss:  0.067 valid_accuracy:  0.396
epoch:  3 loss:  0.0 accuracy:  0.516 valid_loss:  0.066 valid_accuracy:  0.406
epoch:  4 loss:  0.0 accuracy:  0.391 valid_loss:  0.065 valid_accuracy:  0.415
total time spend: 199.8610520362854


In [73]:
def evaluate(epoch, model, testloader):
  test_correct = 0
  test_total = 0
  test_running_loss = 0

  model.eval()
  with torch.no_grad():
    for b in testloader:
      x, y = b.text, b.label
      x, y = x.to(device), y.to(device)
      y_pred = model(x)
      loss = loss_fn(y_pred, y)
      y_pred = torch.argmax(y_pred, dim=1)
      test_correct += (y_pred == y).sum().item()
      test_total += y.size(0)
      test_running_loss += loss.item()

  epoch_test_loss = test_running_loss / len(testloader.dataset)
  epoch_test_acc = test_correct / test_total

  print('epoch: ', epoch,
        'test_loss: ', round(epoch_test_loss, 3),
        'test_accuracy: ', round(epoch_test_acc, 3))
  return epoch_test_loss, epoch_test_acc

In [74]:
epochs = 5
test_loss = []
test_acc = []

for epoch in range(epochs):
  epoch_test_loss, epoch_test_acc =evaluate(epoch, model, test_iterator)
  test_loss.append(epoch_test_loss)
  test_acc.append(epoch_test_acc)
end = time.time()
print(f'total time spend: {end-start}')

epoch:  0 test_loss:  0.016 test_accuracy:  0.402
epoch:  1 test_loss:  0.016 test_accuracy:  0.402
epoch:  2 test_loss:  0.016 test_accuracy:  0.402
epoch:  3 test_loss:  0.016 test_accuracy:  0.402
epoch:  4 test_loss:  0.016 test_accuracy:  0.402
total time spend: 1285.9903943538666
